In [0]:
!mkdir ~/.kaggle2 && echo '{"username":"polinarevina","key":"5bb914194d8b5103c7fab5f72b7c1806"}' >~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c chinese-char-recognition-smmo19 

 99% 183M/185M [00:06<00:00, 23.3MB/s]
100% 185M/185M [00:06<00:00, 28.7MB/s]
 93% 105M/113M [00:01<00:00, 70.2MB/s] 
100% 113M/113M [00:01<00:00, 86.4MB/s]
 94% 184M/195M [00:02<00:00, 80.3MB/s]
100% 195M/195M [00:02<00:00, 96.1MB/s]
 95% 191M/202M [00:02<00:00, 85.2MB/s]
100% 202M/202M [00:02<00:00, 91.1MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 123MB/s]
 95% 157M/165M [00:03<00:00, 55.8MB/s]
100% 165M/165M [00:03<00:00, 49.1MB/s]


In [0]:
!unzip train-4.npy.zip 
!unzip train-3.npy.zip
!unzip test.npy.zip
!unzip train-2.npy.zip
!unzip train-1.npy.zip
!ls

Archive:  train-4.npy.zip
  inflating: train-4.npy             
Archive:  train-3.npy.zip
  inflating: train-3.npy             
Archive:  test.npy.zip
  inflating: test.npy                
Archive:  train-2.npy.zip
  inflating: train-2.npy             
Archive:  train-1.npy.zip
  inflating: train-1.npy             
random_labels.csv  test.npy.zip     train-2.npy      train-3.npy.zip
sample_data	   train-1.npy	    train-2.npy.zip  train-4.npy
test.npy	   train-1.npy.zip  train-3.npy      train-4.npy.zip


In [0]:
!ls

random_labels.csv  test.npy.zip     train-2.npy.zip  train-4.npy.zip
sample_data	   train-1.npy.zip  train-3.npy.zip


In [0]:
import numpy as np 
import tensorflow as tf

In [0]:
data_train = np.load("train-1.npy", allow_pickle = True)
for i in range(2, 5): 
  data_train_n = np.load(f"train-{i}.npy", allow_pickle = True)
  data_train = np.concatenate([data_train, data_train_n])
data_test = np.load("test.npy", allow_pickle = True)

In [0]:
batch_size = 128 
val_size = 0.2

In [0]:
 char_to_id = {char: id for id, char in enumerate (np.unique([char for _, char in data_train]))}
 id_to_char = {id: char for id, char in enumerate (np.unique([char for _, char in data_train]))}
 def train_gen():
    for img, label in data_train[int(len(data_train) * val_size):]:
        img = img[:, :, np.newaxis] # [batch, w, h, channels]
        yield img, char_to_id[label]   
def val_gen():
    for img, label in data_train[:int(len(data_train) * val_size)]:
        img = img[:, :, np.newaxis] # [batch, w, h, channels]
        yield img, char_to_id[label]
def test_gen():
    for img in data_test:
        img = img[:, :, np.newaxis] # [batch, w, h, channels]
        yield img

In [0]:
dataset_train = tf.data.Dataset.from_generator(train_gen,
                                               output_types = (tf.uint8, tf.int32),
                                               output_shapes = ((None, None, 1),()),
                                               )
dataset_val = tf.data.Dataset.from_generator(val_gen,
                                               output_types = (tf.uint8, tf.int32),
                                               output_shapes = ((None, None, 1),()),
                                               )
dataset_test = tf.data.Dataset.from_generator(test_gen,
                                               output_types = tf.uint8,
                                               output_shapes = ((None, None, 1)),
                                               )

In [0]:
def augmentation(img): 
  tf.image.random_brightness(img, 0.2)
  return img 

def preprocess_train(img, label): 
  img = tf.image.resize(img, (64, 64))
  img = tf.cast(img, tf.float32) / 127.5 - 1
  if np.random.rand() < 0.2: 
    img = augmentation(img)
  label = tf.one_hot(label, 1000)
  return img, label 

def preprocess_test(img): 
  img = tf.image.resize(img, (64, 64))
  img = tf.cast(img, tf.float32) / 127.5 - 1
  return img

In [0]:
dataset_train = dataset_train.map(preprocess_train, num_parallel_calls = -1).shuffle(1024).prefetch(-1).batch(batch_size).repeat()
dataset_val = dataset_val.map(preprocess_train, num_parallel_calls = -1).shuffle(1024).prefetch(-1).batch(batch_size).repeat()
dataset_test = dataset_test.map(preprocess_test, num_parallel_calls = -1).prefetch(-1).batch(batch_size)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, LeakyReLU, BatchNormalization
from keras.models import load_model 
from keras.regularizers import l2

Using TensorFlow backend.


In [0]:
initializer = 'lecun_uniform'
def make_model():
    model = Sequential()
    model.add(Conv2D(filters=16, padding='same', kernel_size=(3,3), input_shape=(64,64,1)))
    model.add(LeakyReLU(0.1))
    # model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    # model.add(Dropout(0.5))
    
    model.add(Conv2D(filters=16, padding='same', kernel_size=(3,3), kernel_initializer=initializer))  
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(filters=32, padding='same', kernel_size=(3,3), kernel_initializer=initializer))  
    model.add(LeakyReLU(0.1))
    # model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    # model.add(Dropout(0.3))
    
    model.add(Conv2D(filters=32, padding='same', kernel_size=(3,3), kernel_initializer=initializer))  
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(0.3))

    model.add(Conv2D(filters=64, padding='same', kernel_size=(3,3), kernel_initializer=initializer)) 
    model.add(BatchNormalization()) 
    model.add(LeakyReLU(0.1))
    # model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    # model.add(Dropout(0.2))

    model.add(Conv2D(filters=64, padding='same', kernel_size=(3,3), kernel_initializer=initializer))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer=initializer))  
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))
    # model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    # model.add(Dropout(0.1))

    model.add(Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer=initializer))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=256, padding='same', kernel_size=(3,3), kernel_initializer=initializer))  
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))
    # model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    # model.add(Dropout(0.1))

    model.add(Conv2D(filters=256, padding='same', kernel_size=(3,3), kernel_initializer=initializer))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(LeakyReLU(0.1))
    model.add(Dense(1000))
    model.add(Activation("softmax"))
    return model

In [0]:
model = make_model()

In [0]:
# Сохраним модель
import cv2
import keras
from keras.models import save_model, load_model

class ModelSaveCallback(keras.callbacks.Callback):

    def __init__(self, file_name):
        super(ModelSaveCallback, self).__init__()
        self.file_name = file_name


    def on_epoch_end(self, epoch, logs=None):
        epoch += 1
        model.save(self.file_name.format(epoch))
        print(model.save(self.file_name.format(epoch)))

In [0]:
def load_from_file(model_filename, last_epoch):
  model = load_model(model_filename)
  epoch = last_epoch
  return model

In [0]:
# ?
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001) 

In [0]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [0]:
model.fit(dataset_train, validation_data=dataset_val, epochs=20, steps_per_epoch=1000, validation_steps=200)

Train on 1000 steps, validate on 200 steps
Epoch 1/20
1000/1000 [==============================] - 61s 61ms/step - loss: 0.0188 - acc: 0.9939 - val_loss: 0.0186 - val_acc: 0.9977
Epoch 2/20
1000/1000 [==============================] - 60s 60ms/step - loss: 0.0249 - acc: 0.9921 - val_loss: 0.0209 - val_acc: 0.9975
Epoch 3/20
1000/1000 [==============================] - 60s 60ms/step - loss: 0.0218 - acc: 0.9926 - val_loss: 0.0188 - val_acc: 0.9981
Epoch 4/20
1000/1000 [==============================] - 60s 60ms/step - loss: 0.0211 - acc: 0.9934 - val_loss: 0.0194 - val_acc: 0.9976
Epoch 5/20
1000/1000 [==============================] - 60s 60ms/step - loss: 0.0210 - acc: 0.9933 - val_loss: 0.0199 - val_acc: 0.9976
Epoch 6/20
1000/1000 [==============================] - 61s 61ms/step - loss: 0.0209 - acc: 0.9932 - val_loss: 0.0196 - val_acc: 0.9968
Epoch 7/20
1000/1000 [==============================] - 61s 61ms/step - loss: 0.0212 - acc: 0.9931 - val_loss: 0.0158 - val_acc: 0.9982
Epoch

In [0]:
submission = model.predict(dataset_test) 

In [0]:
submission = submission.argmax(1)
import pandas as pd  
result = pd.read_csv("random_labels.csv") 

In [0]:
result["Category"] = submission 
result["Category"] = result["Category"].apply(lambda x: id_to_char[x])
result.to_csv("submission.csv", index = False) 

In [0]:
! kaggle competitions submit -c chinese-char-recognition-smmo19 -m 'Test' -f submission.csv

100% 965k/965k [00:07<00:00, 140kB/s]
Successfully submitted to Chinese character recognition